In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
import pandas as pd
import os
#from resultsAnalyse import drawConfusionMatrix
import torch
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
def filter_inadequada(example):
    return example["INADEQUADA"] == 0

ds = load_dataset("higopires/RePro-categories-multilabel")
print(len(ds["train"]))
#remove INADEQUADA examples
ds = ds.filter(filter_inadequada)
#remove only keep the firs 1000 examples
#ds["train"] = ds["train"].filter(lambda example, idx: idx < 10, with_indices=True)
#ds["test"] = ds["test"].filter(lambda example, idx: idx < 1000, with_indices=True)
#ds["validation"] = ds["validation"].filter(lambda example, idx: idx < 10, with_indices=True)
print(len(ds["train"]))

8002
7674


In [2]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to probabilities and then to binary predictions
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    print(f"DEBUG: Original logits shape: {logits.shape}")
    print(f"DEBUG: Original labels shape: {labels.shape}")
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    precision = precision_score(labels, predictions, average='micro', zero_division=0)

    recall = recall_score(labels, predictions, average='micro', zero_division=0)
    return {'f1_micro': float(f1), 'precision': float(precision), 'recall': float(recall)}
model_name = "PORTULAN/albertina-900m-portuguese-ptbr-encoder"
tokenizer = AutoTokenizer.from_pretrained(model_name)
def preprocess_function(sample):
    # Tokenize text
    tokenized = tokenizer(sample["review_text"], truncation=True, padding=True, max_length=512, return_tensors="pt")
    
    # Convert binary label columns to a list (e.g., [1, 0, 1, 0, 0, 0])
    labels = []
    for i in range(len(sample["review_text"])):
        label_row = [
            float(sample["ENTREGA"][i]),
            float(sample["OUTROS"][i]),
            float(sample["PRODUTO"][i]),
            float(sample["CONDICOESDERECEBIMENTO"][i]),
            float(sample["ANUNCIO"][i])
        ]
        labels.append(label_row)
    
    tokenized["labels"] = torch.tensor(labels, dtype=torch.float)
    return tokenized
#check if output layer has 5 outputs
#print(model.classifier.out_features)
#model.classifier.out_features = 5  # Explicitly ensure final layer has 5 outputs

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.64M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5,problem_type="multi_label_classification",ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.77G [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at PORTULAN/albertina-900m-portuguese-ptbr-encoder and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model.to(device)
tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/7674 [00:00<?, ? examples/s]

Map:   0%|          | 0/952 [00:00<?, ? examples/s]

Map:   0%|          | 0/966 [00:00<?, ? examples/s]

In [5]:
training_args = TrainingArguments(
    output_dir="./resultsAlbertinaLarge",
    learning_rate=1e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_micro",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


C:\Users\tomas\AppData\Local\Temp\ipykernel_4412\625111805.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [6]:
trainer.train()

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 36.00 MiB. GPU 0 has a total capacity of 12.00 GiB of which 0 bytes is free. Of the allocated memory 16.26 GiB is allocated by PyTorch, and 2.37 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
y_pred = trainer.predict(tokenized_dataset["test"])
y_pred.metrics

DEBUG: Original logits shape: (966, 5)
DEBUG: Original labels shape: (966, 5)


{'test_loss': 0.2603171169757843,
 'test_f1_micro': 0.9301868239921337,
 'test_precision': 0.9304918032786885,
 'test_recall': 0.9298820445609436,
 'test_runtime': 17.4437,
 'test_samples_per_second': 55.378,
 'test_steps_per_second': 11.121}

In [ ]:
trainer.save_model("alberta_large")

In [ ]:
#free cuda memory
torch.cuda.empty_cache()